In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import os
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.tokenize import word_tokenize

In [29]:
for dirname,_,filenames in os.walk('movie-review-sentiment-analysis'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

movie-review-sentiment-analysis\sampleSubmission.csv
movie-review-sentiment-analysis\test.tsv
movie-review-sentiment-analysis\train.tsv


In [30]:
df_train=pd.read_csv(r"movie-review-sentiment-analysis\train.tsv",sep='\t')
df_test=pd.read_csv(r"movie-review-sentiment-analysis\test.tsv",sep='\t')
df_train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [4]:
'''
import csv
with open(r"movie-review-sentiment-analysis\train.tsv") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t")
    for line in tsvreader:
        print(line)
'''

'\nimport csv\nwith open(r"movie-review-sentiment-analysis\train.tsv") as tsvfile:\n    tsvreader = csv.reader(tsvfile, delimiter="\t")\n    for line in tsvreader:\n        print(line)\n'

In [5]:
df_train.isna().sum()

PhraseId      0
SentenceId    0
Phrase        0
Sentiment     0
dtype: int64

In [31]:
lem=WordNetLemmatizer()
stop=set(stopwords.words("english"))
df_train.shape 

(156060, 4)

In [7]:
'''
for i in (df_train['Phrase'][:100]):
    #review=i.split()
    #review=i.lower()
    #review=review.split()
    review=re.sub('[^a-zA-Z]',' ',i)
    review=review.split()
    #review=review.lower()
    #review=re.sub('[^a-zA-Z]',' ',text)
    review=[lem.lemmatize(j) for j in review if not j in stopwords.words('english')]
    review=' '.join(review)
'''

"\nfor i in (df_train['Phrase'][:100]):\n    #review=i.split()\n    #review=i.lower()\n    #review=review.split()\n    review=re.sub('[^a-zA-Z]',' ',i)\n    review=review.split()\n    #review=review.lower()\n    #review=re.sub('[^a-zA-Z]',' ',text)\n    review=[lem.lemmatize(j) for j in review if not j in stopwords.words('english')]\n    review=' '.join(review)\n"

In [32]:
# removing the shit and then stemming it
def prep(text):
    review=re.sub('[^a-zA-Z]',' ',text)
    review=review.lower()
    review=review.split()
    review=[ lem.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    return review

In [33]:
df_train['cleaned_text']=df_train['Phrase'].apply(lambda x: prep(x))

In [34]:
df_test['cleaned_text']=df_test['Phrase'].apply(lambda x: prep(x))

In [35]:
df_train['cleaned_text'][:10]

0    series escapade demonstrating adage good goose...
1       series escapade demonstrating adage good goose
2                                               series
3                                                     
4                                               series
5              escapade demonstrating adage good goose
6                                                     
7              escapade demonstrating adage good goose
8                                             escapade
9                       demonstrating adage good goose
Name: cleaned_text, dtype: object

In [12]:
df_train.Sentiment.value_counts()/len(df_train.Sentiment)

2    0.509945
3    0.210989
1    0.174760
4    0.058990
0    0.045316
Name: Sentiment, dtype: float64

In [89]:
X=df_train.cleaned_text
y=df_train.Sentiment

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix

In [91]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2)

In [92]:
vec=TfidfVectorizer()

In [108]:
vec_test=vec.transform(X_test)

In [112]:
vec_train=vec.fit_transform(X_train)

# Let's see if all the words in the test set occurs in the train set:

In [114]:
cv1 = TfidfVectorizer()
cv1.fit(df_train["Phrase"])

cv2 = TfidfVectorizer()
cv2.fit(df_test["Phrase"])

print("Train Set Vocabulary Size:", len(cv1.vocabulary_))
print("Test Set Vocabulary Size:", len(cv2.vocabulary_))
print("Number of Words that occur in both:", len(set(cv1.vocabulary_.keys()).intersection(set(cv2.vocabulary_.keys()))))

Train Set Vocabulary Size: 15240
Test Set Vocabulary Size: 9546
Number of Words that occur in both: 7057


In [109]:
vec_train.shape,vec_test.shape

((124848, 13598), (31212, 12409))

In [96]:
log=LogisticRegression()

In [110]:
log.fit(vec_train,y_train)
pred_log=log.predict(vec_test)

ValueError: X has 12409 features per sample; expecting 13598

In [28]:
print("accuracy_score",accuracy_score(y_test,pred_log))
print("confusion_matrix\n",confusion_matrix(y_test,pred_log))

accuracy_score 0.6185121107266436
confusion_matrix
 [[  189   695   518    41     2]
 [  114  1674  3372   238     3]
 [   30   688 14265   916    31]
 [    4   148  3389  2778   221]
 [    0    23   413  1061   399]]


# Deep Learning

In [50]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import sequence,text

# Finding number of unique words in train set

In [40]:
all_words=' '.join(X_train)
all_words1=word_tokenize(all_words)
dist=FreqDist(all_words1)
num_unique_word=len(dist)
num_unique_word

13619

# Finding max length of a review in train set

In [41]:
r_len=[]
for text in X_train:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)
    
MAX_REVIEW_LEN=np.max(r_len)
MAX_REVIEW_LEN

30

In [42]:
max_features = num_unique_word
max_words = MAX_REVIEW_LEN
batch_size = 128
epochs = 3
num_classes=5

In [43]:
tokenizer = Tokenizer(num_words=max_features)

In [44]:
tokenizer.fit_on_texts(list(X_train))
X_train1 = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_test)
X_test1 = tokenizer.texts_to_sequences(df_test['cleaned_text'])

In [51]:
X_train1 = sequence.pad_sequences(X_train1, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)
X_test1 = sequence.pad_sequences(X_test1, maxlen=max_words)
print(X_train1.shape,X_val.shape,X_test1.shape)

(124848, 30) (31212, 30) (66292, 30)


# LSTM model

In [54]:
model1=Sequential()
model1.add(Embedding(max_features,100,mask_zero=True))
model1.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model1.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model1.add(Dense(num_classes,activation='softmax'))
model1.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         1361900   
_________________________________________________________________
unified_lstm_2 (UnifiedLSTM) (None, None, 64)          42240     
_________________________________________________________________
unified_lstm_3 (UnifiedLSTM) (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 165       
Total params: 1,416,721
Trainable params: 1,416,721
Non-trainable params: 0
_________________________________________________________________


In [55]:
history1=model1.fit(X_train1, y_train, validation_data=(X_val, y_test),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3
124848/124848 [==============================] - 149s 1ms/sample - loss: 1.0700 - accuracy: 0.5861 - val_loss: 0.8974 - val_accuracy: 0.6388
Epoch 2/3
124848/124848 [==============================] - 154s 1ms/sample - loss: 0.8467 - accuracy: 0.6573 - val_loss: 0.8608 - val_accuracy: 0.6534
Epoch 3/3
124848/124848 [==============================] - 151s 1ms/sample - loss: 0.7916 - accuracy: 0.6795 - val_loss: 0.8557 - val_accuracy: 0.6553


In [56]:
y_pred1=model1.predict_classes(X_test1,verbose=1)

66292/66292 [==============================] - 29s 445us/sample


In [57]:
y_pred1

array([3, 3, 2, ..., 2, 2, 1], dtype=int64)

In [59]:
score, acc = model1.evaluate(X_val, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

31212/31212 [==============================] - 5s 174us/sample - loss: 0.8557 - accuracy: 0.6553
Test score: 0.8556830746599242
Test accuracy: 0.6553249


#  CNN

In [60]:
model2= Sequential()
model2.add(Embedding(max_features,100,input_length=max_words))
model2.add(Dropout(0.2))

model2.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model2.add(GlobalMaxPooling1D())

model2.add(Dense(128,activation='relu'))
model2.add(Dropout(0.2))

model2.add(Dense(num_classes,activation='softmax'))


model2.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 100)           1361900   
_________________________________________________________________
dropout (Dropout)            (None, 30, 100)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 30, 64)            19264     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                

In [61]:
history2=model2.fit(X_train1, y_train, validation_data=(X_val, y_test),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3
124848/124848 [==============================] - 45s 362us/sample - loss: 1.0139 - accuracy: 0.5964 - val_loss: 0.8808 - val_accuracy: 0.6414
Epoch 2/3
124848/124848 [==============================] - 47s 380us/sample - loss: 0.7985 - accuracy: 0.6729 - val_loss: 0.8431 - val_accuracy: 0.6544
Epoch 3/3
124848/124848 [==============================] - 52s 419us/sample - loss: 0.7231 - accuracy: 0.7025 - val_loss: 0.8275 - val_accuracy: 0.6596


In [62]:
y_pred2=model2.predict_classes(X_test1, verbose=1)

66292/66292 [==============================] - 4s 66us/sample


# CNN +GRU

In [63]:
model3= Sequential()
model3.add(Embedding(max_features,100,input_length=max_words))
model3.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.25))
model3.add(GRU(128,return_sequences=True))
model3.add(Dropout(0.3))
model3.add(Flatten())
model3.add(Dense(128,activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(5,activation='softmax'))
model3.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 30, 100)           1361900   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 30, 64)            19264     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 15, 64)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 15, 64)            0         
_________________________________________________________________
unified_gru (UnifiedGRU)     (None, 15, 128)           74496     
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 128)           0         
_________________________________________________________________
flatten (Flatten)            (None, 1920)             

In [64]:
history3=model3.fit(X_train1, y_train, validation_data=(X_val, y_test),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3
124848/124848 [==============================] - 88s 701us/sample - loss: 1.0431 - accuracy: 0.5846 - val_loss: 0.8826 - val_accuracy: 0.6348
Epoch 2/3
124848/124848 [==============================] - 86s 690us/sample - loss: 0.8274 - accuracy: 0.6625 - val_loss: 0.8356 - val_accuracy: 0.6520
Epoch 3/3
124848/124848 [==============================] - 80s 642us/sample - loss: 0.7496 - accuracy: 0.6878 - val_loss: 0.8286 - val_accuracy: 0.6585


In [65]:
y_pred3=model3.predict_classes(X_test1, verbose=1)
y_pred3

66292/66292 [==============================] - 14s 204us/sample


array([3, 3, 2, ..., 1, 1, 1], dtype=int64)

# Bidirectional GRU

In [66]:
model4 = Sequential()

model4.add(Embedding(max_features, 100, input_length=max_words))
model4.add(SpatialDropout1D(0.25))
model4.add(Bidirectional(GRU(128)))
model4.add(Dropout(0.5))

model4.add(Dense(5, activation='softmax'))
model4.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 30, 100)           1361900   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 30, 100)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               176640    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 1285      
Total params: 1,539,825
Trainable params: 1,539,825
Non-trainable params: 0
_________________________________________________________________


In [67]:
history4=model4.fit(X_train1, y_train, validation_data=(X_val, y_test),epochs=epochs, batch_size=batch_size, verbose=1)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3
124848/124848 [==============================] - 150s 1ms/sample - loss: 1.0087 - accuracy: 0.5948 - val_loss: 0.8827 - val_accuracy: 0.6376
Epoch 2/3
124848/124848 [==============================] - 153s 1ms/sample - loss: 0.8245 - accuracy: 0.6618 - val_loss: 0.8445 - val_accuracy: 0.6568
Epoch 3/3
124848/124848 [==============================] - 153s 1ms/sample - loss: 0.7583 - accuracy: 0.6867 - val_loss: 0.8364 - val_accuracy: 0.6620


In [68]:
y_pred4=model4.predict_classes(X_test1, verbose=1)
y_pred4

66292/66292 [==============================] - 28s 421us/sample


array([3, 3, 2, ..., 1, 1, 1], dtype=int64)

# Glove word embedding

In [69]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')
    
def get_embed_mat(EMBEDDING_FILE, max_features,embed_dim):
    # word vectors
    embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE, encoding='utf8'))
    print('Found %s word vectors.' % len(embeddings_index))

    # embedding matrix
    word_index = tokenizer.word_index
    num_words = min(max_features, len(word_index) + 1)
    all_embs = np.stack(embeddings_index.values()) #for random init
    embedding_matrix = np.random.normal(all_embs.mean(), all_embs.std(), 
                                        (num_words, embed_dim))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    max_features = embedding_matrix.shape[0]
    
    return embedding_matrix

In [70]:
# embedding matrix
EMBEDDING_FILE = '../input/glove6b100dtxt/glove.6B.100d.txt'
embed_dim = 100 #word vector dim
embedding_matrix = get_embed_mat(EMBEDDING_FILE,max_features,embed_dim)
print(embedding_matrix.shape)

FileNotFoundError: [Errno 2] No such file or directory: '../input/glove6b100dtxt/glove.6B.100d.txt'

In [ ]:
model5 = Sequential()
model5.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True))
model5.add(SpatialDropout1D(0.25))
model5.add(Bidirectional(GRU(128,return_sequences=True)))
model5.add(Bidirectional(GRU(64,return_sequences=False)))
model5.add(Dropout(0.5))
model5.add(Dense(num_classes, activation='softmax'))
model5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model5.summary()

In [ ]:
history5=model5.fit(X_train1, y_train, validation_data=(X_val, y_test),epochs=4, batch_size=batch_size, verbose=1)

In [ ]:
y_pred5=model5.predict_classes(X_test1, verbose=1)

In [71]:
sub_all=pd.DataFrame({'model1':y_pred1,'model2':y_pred2,'model3':y_pred3,'model4':y_pred4})
pred_mode=sub_all.agg('mode',axis=1)[0].values
sub_all.head()

,model1,model2,model3,model4
0,3,3,3,3
1,3,3,3,3
2,2,2,2,2
3,3,3,3,3
4,3,3,2,3


In [72]:
pred_mode=[int(i) for i in pred_mode]